In [1]:
import numpy as np
import pandas as pd

np.random.seed(42)

n_samples = 200
X1 = np.random.rand(n_samples)
X2 = X1 + np.random.normal(0, 0.01, n_samples)
X3 = 2 * X1 + np.random.normal(0, 0.02, n_samples)
X4 = 3 * X1 + np.random.normal(0, 0.03, n_samples)
X5 = 0.5 * X1 + np.random.normal(0, 0.01, n_samples)
X6 = X1 + X2 + np.random.normal(0, 0.02, n_samples)
X7 = X3 - X5 + np.random.normal(0, 0.02, n_samples)

X = np.column_stack((X1, X2, X3, X4, X5, X6, X7))

y = 5*X1 + 2*X2 - 3*X3 + np.random.normal(0, 0.05, n_samples)

X = (X - X.mean(axis=0)) / X.std(axis=0)

X = np.c_[np.ones(n_samples), X]

def ridge_cost(X, y, w, lam):
    n = len(y)
    y_pred = X.dot(w)
    cost = (1/(2*n)) * np.sum((y_pred - y)**2) + (lam/(2*n)) * np.sum(w[1:]**2)
    return cost

def ridge_gradient(X, y, w, lam):
    n = len(y)
    y_pred = X.dot(w)
    grad = (1/n) * X.T.dot(y_pred - y)
    grad[1:] += (lam/n) * w[1:]
    return grad

def gradient_descent(X, y, lr, lam, epochs=1000):
    w = np.zeros(X.shape[1])
    for _ in range(epochs):
        grad = ridge_gradient(X, y, w, lam)
        w -= lr * grad
    return w

def r2_score(y_true, y_pred):
    ss_res = np.sum((y_true - y_pred)**2)
    ss_tot = np.sum((y_true - np.mean(y_true))**2)
    return 1 - (ss_res / ss_tot)

learning_rates = [0.0001, 0.001, 0.01, 0.1, 1, 10]
lambdas = [1e-15, 1e-10, 1e-5, 1e-3, 0, 1, 10, 20]

results = []

for lr in learning_rates:
    for lam in lambdas:
        w = gradient_descent(X, y, lr, lam, epochs=2000)
        cost = ridge_cost(X, y, w, lam)
        y_pred = X.dot(w)
        r2 = r2_score(y, y_pred)
        results.append((lr, lam, cost, r2))

df_results = pd.DataFrame(results, columns=['Learning Rate', 'Lambda', 'Cost', 'R2_Score'])
best = df_results.loc[df_results['R2_Score'].idxmax()]

print("Best Parameters:")
print(best)


/tmp/ipython-input-4156739715.py:39: RuntimeWarning: invalid value encountered in multiply
  grad[1:] += (lam/n) * w[1:]


Best Parameters:
Learning Rate    1.000000e-01
Lambda           1.000000e-15
Cost             2.440112e-03
R2_Score         9.466391e-01
Name: 24, dtype: float64


In [7]:
!pip install gdown
import gdown
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score

url = "https://drive.google.com/uc?id=1qzCKF6JKKMB0p7ul_lLy8tdmRk3vE_bG"
output = "Hitters.csv"
gdown.download(url, output, quiet=False)

df = pd.read_csv(output)

df = df.dropna(subset=['Salary'])
df = pd.get_dummies(df, columns=['League', 'Division', 'NewLeague'], drop_first=True)

y = df['Salary']
X = df.drop(['Salary', 'Name'], axis=1, errors='ignore')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

lin_reg = LinearRegression()
lin_reg.fit(X_train_scaled, y_train)
y_pred_lin = lin_reg.predict(X_test_scaled)

ridge_reg = Ridge(alpha=0.5748)
ridge_reg.fit(X_train_scaled, y_train)
y_pred_ridge = ridge_reg.predict(X_test_scaled)

lasso_reg = Lasso(alpha=0.5748, max_iter=10000)
lasso_reg.fit(X_train_scaled, y_train)
y_pred_lasso = lasso_reg.predict(X_test_scaled)

results = {}
for name, y_pred in [('Linear', y_pred_lin), ('Ridge', y_pred_ridge), ('Lasso', y_pred_lasso)]:
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    results[name] = {'MSE': mse, 'R2': r2}
    print(f"{name} Regression → MSE: {mse:.2f}, R2: {r2:.4f}")

best_model = max(results, key=lambda m: results[m]['R2'])
print("\nBest Model:", best_model)




FileURLRetrievalError: Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.
	Check FAQ in https://github.com/wkentaro/gdown?tab=readme-ov-file#faq.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1qzCKF6JKKMB0p7ul_lLy8tdmRk3vE_bG

but Gdown can't. Please check connections and permissions.

In [4]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.metrics import mean_squared_error, r2_score


boston = fetch_openml(name='boston', version=1, as_frame=True)
X = boston.data
y = boston.target

print("Dataset loaded successfully!")
print(X.head())

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


alphas = np.logspace(-3, 3, 50)
ridge_cv = RidgeCV(alphas=alphas, scoring='r2', cv=5)
ridge_cv.fit(X_train_scaled, y_train)

ridge_best_alpha = ridge_cv.alpha_
y_pred_ridge = ridge_cv.predict(X_test_scaled)
ridge_mse = mean_squared_error(y_test, y_pred_ridge)
ridge_r2 = r2_score(y_test, y_pred_ridge)

print("\n=== Ridge Regression CV Results ===")
print("Best alpha:", ridge_best_alpha)
print(f"MSE: {ridge_mse:.3f}")
print(f"R²: {ridge_r2:.3f}")


lasso_cv = LassoCV(alphas=alphas, cv=5, max_iter=10000)
lasso_cv.fit(X_train_scaled, y_train)

lasso_best_alpha = lasso_cv.alpha_
y_pred_lasso = lasso_cv.predict(X_test_scaled)
lasso_mse = mean_squared_error(y_test, y_pred_lasso)
lasso_r2 = r2_score(y_test, y_pred_lasso)

print("\n=== Lasso Regression CV Results ===")
print("Best alpha:", lasso_best_alpha)
print(f"MSE: {lasso_mse:.3f}")
print(f"R²: {lasso_r2:.3f}")


print("\n=== Comparison ===")
print(f"Ridge R²: {ridge_r2:.3f}, Lasso R²: {lasso_r2:.3f}")
print(f"Ridge MSE: {ridge_mse:.3f}, Lasso MSE: {lasso_mse:.3f}")

if ridge_r2 > lasso_r2:
    print("Best Model: Ridge Regression (better generalization and stability)")
else:
    print("Best Model: Lasso Regression (more sparse feature selection)")


Dataset loaded successfully!
      CRIM    ZN  INDUS CHAS    NOX     RM   AGE     DIS RAD    TAX  PTRATIO  \
0  0.00632  18.0   2.31    0  0.538  6.575  65.2  4.0900   1  296.0     15.3   
1  0.02731   0.0   7.07    0  0.469  6.421  78.9  4.9671   2  242.0     17.8   
2  0.02729   0.0   7.07    0  0.469  7.185  61.1  4.9671   2  242.0     17.8   
3  0.03237   0.0   2.18    0  0.458  6.998  45.8  6.0622   3  222.0     18.7   
4  0.06905   0.0   2.18    0  0.458  7.147  54.2  6.0622   3  222.0     18.7   

        B  LSTAT  
0  396.90   4.98  
1  396.90   9.14  
2  392.83   4.03  
3  394.63   2.94  
4  396.90   5.33  

=== Ridge Regression CV Results ===
Best alpha: 2.6826957952797246
MSE: 24.349
R²: 0.668

=== Lasso Regression CV Results ===
Best alpha: 0.001
MSE: 24.295
R²: 0.669

=== Comparison ===
Ridge R²: 0.668, Lasso R²: 0.669
Ridge MSE: 24.349, Lasso MSE: 24.295
Best Model: Lasso Regression (more sparse feature selection)


In [5]:

import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score


iris = load_iris()
X = iris.data
y = iris.target

print("Classes:", iris.target_names)
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def compute_cost(X, y, weights):
    n = len(y)
    h = sigmoid(X.dot(weights))
    return -(1/n) * (np.sum(y*np.log(h + 1e-15) + (1-y)*np.log(1-h + 1e-15)))

def gradient_descent(X, y, lr=0.1, epochs=1000):
    n, m = X.shape
    weights = np.zeros(m)
    for _ in range(epochs):
        h = sigmoid(X.dot(weights))
        gradient = (1/n) * X.T.dot(h - y)
        weights -= lr * gradient
    return weights


def train_one_vs_rest(X, y, lr=0.1, epochs=2000):
    classes = np.unique(y)
    all_weights = []
    for c in classes:

        y_binary = np.where(y == c, 1, 0)
        w = gradient_descent(X, y_binary, lr, epochs)
        all_weights.append(w)
    return np.array(all_weights)

X_train_bias = np.c_[np.ones(X_train.shape[0]), X_train]
X_test_bias = np.c_[np.ones(X_test.shape[0]), X_test]

weights = train_one_vs_rest(X_train_bias, y_train, lr=0.1, epochs=2000)


def predict_one_vs_rest(X, weights):
    probs = sigmoid(X.dot(weights.T))
    return np.argmax(probs, axis=1)

y_pred = predict_one_vs_rest(X_test_bias, weights)


accuracy = accuracy_score(y_test, y_pred)
print("Predicted labels:", y_pred)
print("True labels:", y_test)
print(f"\nModel Accuracy: {accuracy*100:.2f}%")


Classes: ['setosa' 'versicolor' 'virginica']
Shape of X: (150, 4)
Shape of y: (150,)
Predicted labels: [1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 2 2 1 1 2 0 2 0 2 2 2 2 2 0 0]
True labels: [1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]

Model Accuracy: 96.67%
